## Imports 

In [7]:
# Imports
from Bio import SeqIO
from Bio.Seq import Seq
import json
import os

## Declares

In [8]:
##

## Main subroutine

In [22]:
# Helper functions
def gard_parser(combo, best_gard, codon_MSA_in):    
    data = [d.strip() for d in open(best_gard).readlines() if "CHARSET" in d]
    coords = []
    if len(data) > 1:
        for pos, i in enumerate(data):
            ## hyphy coordinates are 1 indexed and INCLUSIVE ##
            ## so to get to "true" python coords you need to -1 to start and -1 to stop ##
            ## then you need to +1 to python "stop" coord to get the actual length ## 
            #hyphy_start = i.split(' ')[-1].split(';')[0].split('-')[0]
            #hyphy_stop = i.split(" ")[-1].split(";")[0].split("-")[1]
            #print(f"HYPHY coords for partition {pos +1} --> start {hyphy_start} | stop {hyphy_stop}")
            ## in python (start-1) | (end-1) to get the true coords ##
            ## to get the correct python index[X:Y] (zero index, EXLUSIVE) need to do [start-1,stop] ##
            start = int(i.split(" ")[-1].split(";")[0].split("-")[0])-1
            stop = int(i.split(" ")[-1].split(";")[0].split("-")[1])-1
            coords.append((start,stop))        
        #end for
    ## print out fasta file
    else:
        print(data)
    #end if
    ## now use the coords to pull out seqs in the codon aware MSA ##
    ## write a json for partition and indices associated ## 
    index_data = {}
    for pos, c in enumerate(coords):
        temp_dict = {}
        ## want list of 3 ##
        n = 3
        recs = list(rec for rec in SeqIO.parse(codon_MSA_in, "fasta"))      
        ## actual codons ##
        #codon_map = [list(recs[0].seq)[i*n:(i+1) *n] for i in range((len(list(recs[0].seq)) + n - 1) // n )]
        index_map = [list(range(len(recs[0].seq)))[i*n:(i+1) *n] for i in range((len(list(recs[0].seq)) + n - 1) // n )]
        #print(index_map)
        old_start = c[0]
        old_stop = c[1]
        new_start = ''
        new_stop = ''
        ## getting new start ##
        for imap in index_map:
            if old_start in imap:
                for p, j in enumerate(imap):
                    if old_start == j:
                        if p !=0:
                            new_start = imap[2]+1
                        else:
                            new_start = imap[0]
                        #end if
                    #end if
                #end for
            elif old_stop in imap:
                new_stop = imap[0]
            else:
                continue 
            #end if
        ## sanity check 
        print(f"# Sanity check: OLD {old_start}, {old_stop} | NEW {new_start}, {new_stop} | NEW div by 3 {(int(new_stop)-int(new_start))/3}")
        #print(f"OLD {old_start}, {old_stop} | NEW {new_start}, {new_stop}")
        temp_dict[pos+1] = [i+1 for i in list(range(int(new_start),int(new_stop)))]
        #print(temp_dict)
        index_data.update(temp_dict)
        
        ### ~ CHANGE TO PLACE YOU WANT OUTPUT ~ ###
        codon_out = os.path.join("..", "results", label, "%s.%s.codon.fas" % (combo, str(pos+1)))
        
        print("# Saving partition to:", codon_out)
        with open(codon_out, "w") as out_f:
            ## just grabbing one record right now ## 
            #for record in recs[0:1]:
            for record in recs:
                ## now grabbing actual partition ##
                partition = record[int(new_start):int(new_stop)]
                ## double sanity check ##
                #print(len(partition),len(partition)/3)
                #print(partition.seq)
                out_f.write(">{}\n{}\n".format(partition.id, partition.seq))
            #end for
        #end with
        
    #print(index_data)
    ## write json here ## 
    #json_title = "CATS_8-2021/CATS_8-20.codons/FCOV1_%s.PARTITION_info.json" % (combo)
    #with open(json_title, 'w') as json_file:
    #      json.dump(index_data, json_file, indent=4)
    #temp = {}
    #temp[combo] = list(index_data.keys())
    ##print(temp)
    print(f"\nDone with {combo}\n")
    #return temp

In [23]:


label =  snakemake.params.genelabel

# mammalian_REV3L_codons.SA.fasta.best-gard
best_gard = os.path.join("..", "results", label, label + "_codons.SA.fasta.best-gard")
input_msa = os.path.join("..", "results", label, label + "_codons.SA.fasta")


print("# Processing best-gard:", best_gard)
print("# Input Alignment:", input_msa)

gard_parser(label, best_gard, input_msa)


# Processing best-gard: ../results/mammalian_REV3L/mammalian_REV3L_codons.SA.fasta.best-gard
# Input Alignment: ../results/mammalian_REV3L/mammalian_REV3L_codons.SA.fasta
# Sanity check: OLD 0, 9059 | NEW 0, 9057 | NEW div by 3 3019.0
# Saving partition to: ../results/mammalian_REV3L/mammalian_REV3L.1.codon.fas
# Sanity check: OLD 9060, 10883 | NEW 9060, 10881 | NEW div by 3 607.0
# Saving partition to: ../results/mammalian_REV3L/mammalian_REV3L.2.codon.fas

Done with mammalian_REV3L



## End of file